In [7]:
import json
with open(r"/Users/cylis/MachineLearning/ml_course/June_Labeling/Data/2022-01-01/581tweet.txt",'r') as file_in:
    for line in file_in:
        tweet = json.loads(line)
        text = tweet['text']
        print(text)
        break

RT @BOSS_JAEHYUN: Jaehyun and Jaemin’s high five 🖐 https://t.co/D9XnbQQJEq


In [8]:
with open('text','w',encoding='utf-8') as file_out:
    file_out.write(text)

In [9]:
import re
import time
from datetime import datetime

url_pattern = re.compile(r'http\S+')
hashtag_pattern = re.compile(r'#\S+')
mention_pattern = re.compile(r'@\S+')
rule = re.compile(r'[^a-zA-Z0-9\s]')
# rule = re.compile(r'[^a-zA-Z\s]')
rule_all = re.compile(r'[^a-zA-Z0-9\s]')


class User():
    '''
    TwitterUser class can used to save a user
    Including userbased features
    '''
    #user_feature = []
    def __init__(self, tweet_json):
        self.user_json = tweet_json['user']
        self.id_str = self.user_json['id_str']
        self.screen_name = self.user_json['screen_name']
        self.name = self.user_json['name']
        self.created_at = self.json_date_to_stamp(self.user_json['created_at'])
        self.followers_count = self.get_followers_count()
        self.friends_count = self.get_friends_count()
        self.statuses_count = self.get_statuses_count()

    def json_date_to_stamp(self, json_date):
        '''
        exchange date from json format to timestamp(int)
        input:
            date from json
        output:
            int
        '''
        time_strpt = time.strptime(json_date, '%a %b %d %H:%M:%S +0000 %Y')
        stamp = int(time.mktime(time_strpt))
        return stamp

    def json_date_to_os(self, json_date):
        '''
        exchange date from json format to linux OS format
        input:
            date from json
        output:
            datetime
        '''
        time_strpt = time.strftime('%Y-%m-%d %H:%M:%S',
                                   time.strptime(json_date, '%a %b %d %H:%M:%S +0000 %Y'))
        os_time = datetime.strptime(str(time_strpt), '%Y-%m-%d %H:%M:%S')
        return os_time
    
    def get_screen_name(self):
        '''
        return user screen name
        '''
        return self.screen_name

    def get_followers_count(self):
        '''
        return follower count
        '''
        followers_count = self.user_json['followers_count']
        if followers_count == 0:
            followers_count = 1
        return followers_count

    def get_friends_count(self):
        '''
        return friends count
        '''
        friends_count = self.user_json['friends_count']
        if friends_count == 0:
            friends_count = 1
        return friends_count

    def get_statuses_count(self):
        '''
        return statuses count
        '''
        statuses_count = self.user_json['statuses_count']
        if statuses_count == 0:
            statuses_count = 1
        return statuses_count

    def get_user_age(self):
        '''
        Age of an account
        get age feature of an account, remember call this function all the time. Time exchange
        '''
        account_start_time = self.json_date_to_os(self.user_json['created_at'])
        now_time = datetime.now()
        account_age = (now_time-account_start_time).days
        if account_age == 0:
            account_age = 1
        return account_age

    def get_user_favourites(self):
        '''
        get user favourites count
        '''
        favourites_count = self.user_json['favourites_count']
        return favourites_count

    def get_user_lists(self):
        '''
        get user lists
        '''
        listed_count = self.user_json['listed_count']
        return listed_count
    
    def get_description_len(self):
        desc_text = self.user_json['description']
        # filter_text = self.text_filter(desc_text)
        if desc_text is None:
            return 0
        else:
            return len(desc_text)

    def text_filter(self, text):
        # filter email
        new_text = re.sub(url_pattern,' ',text)

        # filter hashtag
        #new_text = re.sub(hashtag_pattern,'',new_text)

        # filter mention
        new_text = re.sub(mention_pattern,' ',new_text)

        # . _ - link words together
        new_text = re.sub('\.|_|-', '', new_text)

        # keep
        new_text = re.sub(rule_all, ' ',new_text)
        # new_text = re.sub(rule, ' ',new_text)

        # filter \t \n
        new_text = re.sub('\t|\n|\r|\v',' ', new_text)
        # filter multi spaces
        new_text = re.sub(' +', ' ', new_text)

        # remove space of head or tail
        new_text = re.sub('^ | $','', new_text)

        return new_text

    def get_ratio_follow_friend(self):
        return self.get_followers_count()/self.get_friends_count()


class Tweet():
    '''
    TwitterTweet class can used to save a tweet
    '''
    def __init__(self, tweet_json):
        self.tweet_json = tweet_json
        self.user = User(tweet_json)


        self.text = tweet_json['text']
        self.timestr = self.json_date_to_stamp(tweet_json['created_at'])
        self.tid_str = self.tweet_json['id_str']

    def json_date_to_stamp(self, json_date):
        '''
        exchange date from json format to timestamp(int)
        input:
            date from json
        output:
            int
        '''
        time_strpt = time.strptime(json_date, '%a %b %d %H:%M:%S +0000 %Y')
        stamp = int(time.mktime(time_strpt))
        return stamp
    
    def get_id(self):
        return self.tid_str

    def is_en(self):
        return self.tweet_json["lang"]=="en"

    def get_create_at(self):
        '''
        get create at
        '''
        return self.timestr

    def get_retweet_count(self):
        '''
        get retweet count
        '''
        retweet_count = self.tweet_json['retweet_count']
        return retweet_count

    def get_hashtag_count(self):
        '''
        get number of hashtags
        '''
        hashtags = self.tweet_json['entities']['hashtags']
        return len(hashtags)

    def get_mention_count(self):
        '''
        get number of hashtags
        '''
        mentions = self.tweet_json['entities']['user_mentions']
        return len(mentions)

    def get_url_count(self):
        '''
        get number of urls
        '''
        urls = self.tweet_json['entities']['urls']
        return len(urls)

    def get_text_len(self):
        '''
        return chars of text
        '''
        return len(self.text)

    def get_text_digits(self):
        '''
        return number of digits in tweet
        '''
        count_digits = 0
        for ch in self.text:
            if ord(ch)>=48 and ord(ch)<=57:
                count_digits = count_digits+1
        return count_digits

    def get_tweet_features(self):
        feature_list = []
        feature_list.append(self.user.get_user_age())           # 1
        feature_list.append(self.user.get_description_len())    # 2
        feature_list.append(self.user.get_followers_count())    # 3
        feature_list.append(self.user.get_friends_count())      # 4
        feature_list.append(self.user.get_user_favourites())    # 5
        feature_list.append(self.user.get_user_lists())         # 6
        feature_list.append(self.user.get_statuses_count())     # 7
        feature_list.append(self.get_hashtag_count())           # 8
        feature_list.append(self.get_mention_count())           # 9
        feature_list.append(self.get_url_count())               # 10
        feature_list.append(self.get_text_len())                # 11
        feature_list.append(self.get_text_digits())             # 12

        return feature_list

In [13]:
import os
import json
import datetime
from tool.Twitter import Tweet

start_date = "20201117"
end_date = "20210521"
start_date_datetime = datetime.datetime.strptime(start_date, '%Y%m%d')
end_date_datetime = datetime.datetime.strptime(end_date, '%Y%m%d')
proc_date = start_date_datetime
duration = 300   # t

data_check_list = os.listdir("Data/")
data_check_dic = {i:1 for i in data_check_list}

for _ in range(duration):
    # process the data in this date
    proc_date_str = proc_date.strftime("%Y-%m-%d")

    input_data_folder_path = "Data/"+proc_date_str+"/"
    output_data_folder_path =   "Tmp/"+proc_date_str+"/"


    if not proc_date_str in data_check_dic.keys():
        proc_date = proc_date+datetime.timedelta(days=1)
        if proc_date == end_date_datetime:
            break
        continue
    
    if not os.path.exists(output_data_folder_path):
        os.makedirs(output_data_folder_path)

    output_data_file = output_data_folder_path+"tweet_feature"
    with open(output_data_file, 'w', encoding='utf-8') as file_out:


        for filename in os.listdir(input_data_folder_path):
            input_data_path = input_data_folder_path+filename

            with open(input_data_path, 'r', encoding='utf-8', errors='ignore') as file_in:
                
                for line in file_in:
                    try:
                        tweet = json.loads(line)
                        tweet_obj = Tweet(tweet)
                    except:
                        print(tweet)
                        continue
                    
                    if not tweet_obj.is_en():
                        continue

                    tweet_id = tweet_obj.get_id()
                    user_id = tweet_obj.user.id_str
                    feature_list = tweet_obj.get_tweet_features()
                    file_out.write(tweet_id+'\t'+user_id+'\t')

                    for f in feature_list:
                        file_out.write(str(f))
                        file_out.write('\t')
                    file_out.write('\n')
                    file_out.flush()
                    
    proc_date = proc_date+datetime.timedelta(days=1)
    if proc_date == end_date_datetime:
        break
    

ModuleNotFoundError: No module named 'context_locals'

In [14]:
import os
import sys
import random
import datetime
import binascii

start_date = "20220101"
end_date = "20220102"
start_date_datetime = datetime.datetime.strptime(start_date, '%Y%m%d')
end_date_datetime = datetime.datetime.strptime(end_date, '%Y%m%d')
proc_date = start_date_datetime
duration = 300   #
future_days = 5

data_check_list = os.listdir("Data/")
data_check_dic = {i:1 for i in data_check_list}

for _ in range(duration):
    # process the data in this date
    proc_date_str = proc_date.strftime("%Y-%m-%d")

    print(proc_date_str)

    input_data_folder_path = "Tmp/"+proc_date_str+"/"
    output_data_folder_path =   "Tmp/"+proc_date_str+"/"

    if not proc_date_str in data_check_dic.keys():
        proc_date = proc_date+datetime.timedelta(days=1)
        if proc_date == end_date_datetime:
            break
        continue

    input_data_file_list = []
    for i in range(future_days):
        future_date =  proc_date+datetime.timedelta(days=i)
        future_date_str = future_date.strftime("%Y-%m-%d")

        if not future_date_str in data_check_dic.keys():
            continue
        future_data_folder_path = "Tmp/"+future_date_str+"/"
        future_data_file = future_data_folder_path+"tweet_text"
        input_data_file_list.append(future_data_file)


    numtweets = 0
    numHashes = 10
    curtweets = 0

    print('Shingling tweets...')

    # The current shingle ID value to assign to the next new shingle we 
    # encounter. When a shingle gets added to the dictionary, we'll increment this
    # value.
    curShingleID = 0

    # Create a dictionary of the articles, mapping the article identifier (e.g., 
    # "t8470") to the list of shingle IDs that appear in the document.
    tweetAsShingleSets = {}
    tweetNames = []
    totalShingles = 0

    # if not os.path.exists(output_data_folder_path):
    #     os.makedirs(output_data_folder_path)

    # tweet_limit = 10000
    for input_data_file in input_data_file_list:
        with open(input_data_file, 'r', encoding='utf-8', errors='ignore') as file_in:
            for line in file_in:
                # tweet_limit -= 1
                # if tweet_limit < 0:
                #     break
                
                tweet_id = line.strip().split('\t')[0]
                try:
                    tweet_text =  line.strip().split('\t')[2]
                except:
                    continue
                tweet_token = tweet_text.strip().split()
                
                # Maintain a list of all tweet IDs.
                tweetNames.append(tweet_id)

                curtweets = curtweets + 1

                # 'shinglesInDoc' will hold all of the unique shingle IDs present in the 
                # current document. If a shingle ID occurs multiple times in the document,
                # it will only appear once in the set (this is a property of Python sets).
                shinglesInTweet = set()

                # For each word in the document...
                for index in range(0, len(tweet_token) - 2):
                    # Construct the shingle text by combining three words together.
                    shingle =tweet_token[index] + ' ' + tweet_token[index + 1] + ' ' + tweet_token[index + 2]
                    # shingle = tweet_text[index] + ' ' + tweet_text[index + 1] + ' ' + tweet_text[index + 2]
    

                    # Hash the shingle to a 32-bit integer.
                    crc = binascii.crc32(bytes(shingle, encoding="ascii")) & 0xffffffff

            
                    # Add the hash value to the list of shingles for the current document. 
                    # Note that set objects will only add the value to the set if the set 
                    # doesn't already contain it. 
                    shinglesInTweet.add(crc)

                # Store the completed list of shingles for this document in the dictionary.
                tweetAsShingleSets[tweet_id] = shinglesInTweet
    numtweets = curtweets


    # =============================================================================
    #                     Define Triangle Matrices
    # =============================================================================

    # Define virtual Triangle matrices to hold the similarity values. For storing
    # similarities between pairs, we only need roughly half the elements of a full
    # matrix. Using a triangle matrix requires less than half the memory of a full
    # matrix, and can protect the programmer from inadvertently accessing one of
    # the empty/invalid cells of a full matrix.

    # Calculate the number of elements needed in our triangle matrix
    numElems = int(numtweets * (numtweets - 1) / 2)

    # Initialize two empty lists to store the similarity values. 
    # 'JSim' will be for the actual Jaccard Similarity values. 
    # 'estJSim' will be for the estimated Jaccard Similarities found by comparing
    # the MinHash signatures.
    JSim = [0 for x in range(numElems)]
    estJSim = [0 for x in range(numElems)]

    def getTriangleIndex(i, j):
    # If i == j that's an error.
        if i == j:
            sys.stderr.write("Can't access triangle matrix with i == j")
            sys.exit(1)
        # If j < i just swap the values.
        if j < i:
            i, j = j, i
            # temp = i
            # i = j
            # j = temp

        # Calculate the index within the triangular array.
        # This fancy indexing scheme is taken from pg. 211 of:
        # http://infolab.stanford.edu/~ullman/mmds/ch6.pdf
        # But I adapted it for a 0-based index.
        # Note: The division by two should not truncate, it
        #       needs to be a float. 
        k = int(i * (numtweets - (i + 1) / 2.0) + j - i) - 1

        return k

    # =============================================================================
    #                 Generate MinHash Signatures
    # =============================================================================
    print ('Generating random hash functions...')

    # Record the maximum shingle ID that we assigned.
    maxShingleID = 2**32-1

    # We need the next largest prime number above 'maxShingleID'.
    # I looked this value up here: 
    # http://compoasso.free.fr/primelistweb/page/prime/liste_online_en.php
    nextPrime = 4294967311


    # Our random hash function will take the form of:
    #   h(x) = (a*x + b) % c
    # Where 'x' is the input value, 'a' and 'b' are random coefficients, and 'c' is
    # a prime number just greater than maxShingleID.

    # Generate a list of 'k' random coefficients for the random hash functions,
    # while ensuring that the same value does not appear multiple times in the 
    # list.
    def pickRandomCoeffs(k):
        # Create a list of 'k' random values.
        randList = []
    
        while k > 0:
            # Get a random shingle ID.
            randIndex = random.randint(0, maxShingleID) 
    
            # Ensure that each random number is unique.
            while randIndex in randList:
                randIndex = random.randint(0, maxShingleID) 
        
            # Add the random number to the list.
            randList.append(randIndex)
            k = k - 1
        
        return randList

    # For each of the 'numHashes' hash functions, generate a different coefficient 'a' and 'b'.   
    coeffA = pickRandomCoeffs(numHashes)
    coeffB = pickRandomCoeffs(numHashes)


    print ('Generating MinHash signatures for all documents...')

    # List of documents represented as signature vectors
    signatures = []

    # Rather than generating a random permutation of all possible shingles, 
    # we'll just hash the IDs of the shingles that are *actually in the document*,
    # then take the lowest resulting hash code value. This corresponds to the index 
    # of the first shingle that you would have encountered in the random order.

    # For each tweet...
    for tweetID in tweetNames:
    
        # Get the shingle set for this document.
        shingleIDSet = tweetAsShingleSets[tweetID]
    
        # The resulting minhash signature for this document. 
        signature = []
    
        # For each of the random hash functions...
        for i in range(0, numHashes):
        
            # For each of the shingles actually in the document, calculate its hash code
            # using hash function 'i'. 
        
            # Track the lowest hash ID seen. Initialize 'minHashCode' to be greater than
            # the maximum possible value output by the hash.
            minHashCode = nextPrime + 1
        
            # For each shingle in the document...
            for shingleID in shingleIDSet:
                # Evaluate the hash function.
                hashCode = (coeffA[i] * shingleID + coeffB[i]) % nextPrime 
        
                # Track the lowest hash code seen.
                if hashCode < minHashCode:
                    minHashCode = hashCode

            # Add the smallest hash code value as component number 'i' of the signature.
            signature.append(minHashCode)
        # Store the MinHash signature for this document.
        signatures.append(signature)


    # =============================================================================
    #                     Compare All Signatures
    # =============================================================================  
    print ('Comparing all signatures...')

    # Creates a N x N matrix initialized to 0.
    # For each of the test documents...
    for i in range(0, numtweets):
   
        # Get the MinHash signature for document i.
        signature1 = signatures[i]
        # if i%100==0:
        #     print(i)
        
        # For each of the other test documents...
        for j in range(i + 1, numtweets):
        
            # Get the MinHash signature for document j.
            try:
                signature2 = signatures[j]
            except:
                print("warning on comparing signatures")
                continue
        
            count = 0
            # Count the number of positions in the minhash signature which are equal.
            for k in range(0, numHashes):
                count = count + (signature1[k] == signature2[k])
        
            # Record the percentage of positions which matched.    
            estJSim[getTriangleIndex(i, j)] = (count / numHashes)


    # =============================================================================
    #                   Display Similar Document Pairs
    # =============================================================================  




    threshold = 0.5
    # For each of the document pairs...
    with open(output_data_folder_path+'cluster','w') as same_file, open(output_data_folder_path+'non_cluster','w') as unique_file:
        same_dict = {}
        same_list = []
        for i in range(0, numtweets):
            if not tweetNames[i] in same_list:
                value = ''
                for j in range(i + 1, numtweets):
                    if not tweetNames[j] in same_list:
                        # Retrieve the estimated similarity value for this pair.
                        estJ = estJSim[getTriangleIndex(i, j)]
                
                        # If the similarity is above the threshold...
                        if estJ > threshold:

                            if tweetNames[i] in same_dict.keys():
                                value = same_dict[tweetNames[i]]+':'+str(tweetNames[j])
                                same_dict[tweetNames[i]] = value
                                same_list.append(tweetNames[j])
                            else:
                                value = str(tweetNames[j])
                                same_dict[tweetNames[i]] = value
                                same_list.append(tweetNames[i])
                                same_list.append(tweetNames[j])
                # unique
                if not value:
                    unique_file.write(str(tweetNames[i])+':')             
                            

        
        for k, v in same_dict.items():
            same_file.write(k+':'+v+'\n')


    proc_date = proc_date+datetime.timedelta(days=1)
    if proc_date == end_date_datetime:
        break

2022-01-01
Shingling tweets...


FileNotFoundError: [Errno 2] No such file or directory: 'Tmp/2022-01-01/tweet_text'

In [15]:
import os
import re
import json
import datetime
# from nltk.corpus import stopwords

url_pattern = re.compile(r'http\S+')
hashtag_pattern = re.compile(r'#\S+')
mention_pattern = re.compile(r'@\S+')
rule = re.compile(r'[^a-zA-Z\s]')


def extract_text(tw):
    if is_extended(tw):
        return tw["extended_tweet"]["full_text"]
    else:
        return tw["text"]

def is_en(tw):
    return tw["lang"]=="en"


def get_tweet_id(tw):
    return tw["id_str"]

def get_user_id(tw):
    return tw["user"]["id_str"]

def is_extended(tw):
    if "extended_tweet" in tw:
        return True
    else:
        return False    

def text_filter(text):
    if text == None:
        return ''

    new_text = text

    
    #filter url
    new_text = re.sub(url_pattern,'',new_text)

    #filter hashtag
    new_text = re.sub(hashtag_pattern,'',new_text)

    #filter mention
    new_text = re.sub(mention_pattern,' ',new_text)

    # filter \t \n
    new_text = re.sub('\t|\n|\r|\v',' ', new_text)
    # filter multi spaces
    new_text = re.sub(' +', ' ', new_text)

    # remove space of head or tail
    new_text = re.sub('^ | $','', new_text)

    # keep number and char
    new_text = re.sub(rule, '', new_text)
    return new_text


start_date = "20201117"
end_date = "20210521"
start_date_datetime = datetime.datetime.strptime(start_date, '%Y%m%d')
end_date_datetime = datetime.datetime.strptime(end_date, '%Y%m%d')
proc_date = start_date_datetime
duration = 300   # t

data_check_list = os.listdir("Data/")
data_check_dic = {i:1 for i in data_check_list}

for _ in range(duration):
    # process the data in this date
    proc_date_str = proc_date.strftime("%Y-%m-%d")

    input_data_folder_path = "Data/"+proc_date_str+"/"
    output_data_folder_path =   "Tmp/"+proc_date_str+"/"


    if not proc_date_str in data_check_dic.keys():
        proc_date = proc_date+datetime.timedelta(days=1)
        if proc_date == end_date_datetime:
            break
        continue
    
    if not os.path.exists(output_data_folder_path):
        os.makedirs(output_data_folder_path)

    output_data_file = output_data_folder_path+"tweet_text"
    with open(output_data_file, 'w', encoding='utf-8') as file_out:


        for filename in os.listdir(input_data_folder_path):
            input_data_path = input_data_folder_path+filename

            with open(input_data_path, 'r', encoding='utf-8', errors='ignore') as file_in:
                
        
                for line in file_in:
                    try:
                        tweet = json.loads(line)
                    except:
                        print(tweet)
                        continue
                    if is_en(tweet):
                        tweet_id = get_tweet_id(tweet)
                        user_id = get_user_id(tweet)
                        text = extract_text(tweet)
                        text = text_filter(text)
                        # if len(text) < 20:
                        #     continue
                        file_out.write(tweet_id+'\t'+user_id+'\t'+text+'\t')
                        file_out.write('\n')
                        file_out.flush()


    proc_date = proc_date+datetime.timedelta(days=1)
    if proc_date == end_date_datetime:
        break





In [16]:
import os
import json
import datetime




start_date = "20220101"
end_date = "20220102"
start_date_datetime = datetime.datetime.strptime(start_date, '%Y%m%d')
end_date_datetime = datetime.datetime.strptime(end_date, '%Y%m%d')
proc_date = start_date_datetime
duration = 300   #

data_check_list = os.listdir("Data/")
data_check_dic = {i:1 for i in data_check_list}


for _ in range(duration):
    # process the data in this date
    proc_date_str = proc_date.strftime("%Y-%m-%d")

    input_data_tmp_path = "Tmp/"+proc_date_str+"/"
    input_data_folder_path = "Cluster/"+proc_date_str+"/"
    output_data_folder_path =  "Label/"+proc_date_str+"/"

    if not proc_date_str in data_check_dic.keys():
        proc_date = proc_date+datetime.timedelta(days=1)
        if proc_date == end_date_datetime:
            break
        continue

    if not os.path.exists(output_data_folder_path):
        os.makedirs(output_data_folder_path)

    input_data_text = input_data_tmp_path+"tweet_text"
    input_data_cluster = input_data_tmp_path+'cluster'
    input_data_label = input_data_folder_path+'tweets.txt'
    output_data_file = output_data_folder_path+"labeled_tweets.txt"

    cluster_reverse = {}
    with open(input_data_cluster, 'r') as file_in:
        for line in file_in:
            line_split = line.strip().split(":")
            for lc in line_split:
                cluster_reverse[lc] = line_split[0]

    label_dic = {}
    with open(input_data_label, 'r') as file_in:
        for line in file_in:
            line_split = line.strip().split("\t")
            if not line_split[0] == 'N':
                label_dic[line_split[2]] = line_split[0]

    with open(input_data_text, 'r') as file_in, \
        open(output_data_file, 'w') as file_out:
        for line in file_in:
            line_split = line.strip().split("\t")
            tweet_id = line_split[0]
            user_id = line_split[1]
            tweet_text = line_split[2]

            if not tweet_id in cluster_reverse.keys():
                continue
            cluster_id = cluster_reverse[tweet_id]

            if not cluster_id in label_dic.keys():
                continue

            label = label_dic[cluster_id]
      

            file_out.write(tweet_id+'\t')
            file_out.write(user_id+'\t')
            file_out.write(tweet_text+'\t')
            file_out.write(label+'\n')
            file_out.flush()

    proc_date = proc_date+datetime.timedelta(days=1)
    if proc_date == end_date_datetime:
        break



FileNotFoundError: [Errno 2] No such file or directory: 'Tmp/2022-01-01/cluster'

In [18]:
import os
import json
import datetime
from tool.Twitter import Tweet

start_date = "20220101"
end_date = "20220102"
start_date_datetime = datetime.datetime.strptime(start_date, '%Y%m%d')
end_date_datetime = datetime.datetime.strptime(end_date, '%Y%m%d')
proc_date = start_date_datetime
duration = 300   # t

data_check_list = os.listdir("Data/")
data_check_dic = {i:1 for i in data_check_list}

for _ in range(duration):
    # process the data in this date
    proc_date_str = proc_date.strftime("%Y-%m-%d")

    input_data_folder_path = "Data/"+proc_date_str+"/"
    output_data_folder_path =   "Tmp/"+proc_date_str+"/"


    if not proc_date_str in data_check_dic.keys():
        proc_date = proc_date+datetime.timedelta(days=1)
        if proc_date == end_date_datetime:
            break
        continue
    
    if not os.path.exists(output_data_folder_path):
        os.makedirs(output_data_folder_path)

    output_data_file = output_data_folder_path+"tweet_feature"
    with open(output_data_file, 'w', encoding='utf-8') as file_out:


        for filename in os.listdir(input_data_folder_path):
            input_data_path = input_data_folder_path+filename

            with open(input_data_path, 'r', encoding='utf-8', errors='ignore') as file_in:
                
                for line in file_in:
                    try:
                        tweet = json.loads(line)
                        tweet_obj = Tweet(tweet)
                    except:
                        print(tweet)
                        continue
                    
                    if not tweet_obj.is_en():
                        continue

                    tweet_id = tweet_obj.get_id()
                    user_id = tweet_obj.user.id_str
                    feature_list = tweet_obj.get_tweet_features()
                    file_out.write(tweet_id+'\t'+user_id+'\t')

                    for f in feature_list:
                        file_out.write(str(f))
                        file_out.write('\t')
                    file_out.write('\n')
                    file_out.flush()
                    
    proc_date = proc_date+datetime.timedelta(days=1)
    if proc_date == end_date_datetime:
        break

IsADirectoryError: [Errno 21] Is a directory: 'Data/2022-01-01/.ipynb_checkpoints'

In [20]:
import os
import re
import json
import datetime
# from nltk.corpus import stopwords

url_pattern = re.compile(r'http\S+')
hashtag_pattern = re.compile(r'#\S+')
mention_pattern = re.compile(r'@\S+')
rule = re.compile(r'[^a-zA-Z\s]')


def extract_text(tw):
    if is_extended(tw):
        return tw["extended_tweet"]["full_text"]
    else:
        return tw["text"]

def is_en(tw):
    return tw["lang"]=="en"


def get_tweet_id(tw):
    return tw["id_str"]

def get_user_id(tw):
    return tw["user"]["id_str"]

def is_extended(tw):
    if "extended_tweet" in tw:
        return True
    else:
        return False    

def text_filter(text):
    if text == None:
        return ''

    new_text = text

    
    #filter url
    new_text = re.sub(url_pattern,'',new_text)

    #filter hashtag
    new_text = re.sub(hashtag_pattern,'',new_text)

    #filter mention
    new_text = re.sub(mention_pattern,' ',new_text)

    # filter \t \n
    new_text = re.sub('\t|\n|\r|\v',' ', new_text)
    # filter multi spaces
    new_text = re.sub(' +', ' ', new_text)

    # remove space of head or tail
    new_text = re.sub('^ | $','', new_text)

    # keep number and char
    new_text = re.sub(rule, '', new_text)
    return new_text


start_date = "20220101"
end_date = "20220102"
start_date_datetime = datetime.datetime.strptime(start_date, '%Y%m%d')
end_date_datetime = datetime.datetime.strptime(end_date, '%Y%m%d')
proc_date = start_date_datetime
duration = 300   # t

data_check_list = os.listdir("Data/")
data_check_dic = {i:1 for i in data_check_list}

for _ in range(duration):
    # process the data in this date
    proc_date_str = proc_date.strftime("%Y-%m-%d")

    input_data_folder_path = "Data/"+proc_date_str+"/"
    output_data_folder_path =   "Tmp/"+proc_date_str+"/"


    if not proc_date_str in data_check_dic.keys():
        proc_date = proc_date+datetime.timedelta(days=1)
        if proc_date == end_date_datetime:
            break
        continue
    
    if not os.path.exists(output_data_folder_path):
        os.makedirs(output_data_folder_path)

    output_data_file = output_data_folder_path+"tweet_text"
    with open(output_data_file, 'w', encoding='utf-8') as file_out:


        for filename in os.listdir(input_data_folder_path):
            input_data_path = input_data_folder_path+filename

            with open(input_data_path, 'r', encoding='utf-8', errors='ignore') as file_in:
                
        
                for line in file_in:
                    try:
                        tweet = json.loads(line)
                    except:
                        print(tweet)
                        continue
                    if is_en(tweet):
                        tweet_id = get_tweet_id(tweet)
                        user_id = get_user_id(tweet)
                        text = extract_text(tweet)
                        text = text_filter(text)
                        # if len(text) < 20:
                        #     continue
                        file_out.write(tweet_id+'\t'+user_id+'\t'+text+'\t')
                        file_out.write('\n')
                        file_out.flush()


    proc_date = proc_date+datetime.timedelta(days=1)
    if proc_date == end_date_datetime:
        break





IsADirectoryError: [Errno 21] Is a directory: 'Data/2022-01-01/.ipynb_checkpoints'

In [21]:
import os
import sys
import random
import datetime
import binascii




start_date = "20220101"
end_date = "20220102"
start_date_datetime = datetime.datetime.strptime(start_date, '%Y%m%d')
end_date_datetime = datetime.datetime.strptime(end_date, '%Y%m%d')
proc_date = start_date_datetime
duration = 300   #
future_days = 5

data_check_list = os.listdir("Data/")
data_check_dic = {i:1 for i in data_check_list}

for _ in range(duration):
    # process the data in this date
    proc_date_str = proc_date.strftime("%Y-%m-%d")

    print(proc_date_str)

    input_data_folder_path = "Tmp/"+proc_date_str+"/"
    output_data_folder_path =   "Tmp/"+proc_date_str+"/"

    if not proc_date_str in data_check_dic.keys():
        proc_date = proc_date+datetime.timedelta(days=1)
        if proc_date == end_date_datetime:
            break
        continue

    input_data_file_list = []
    for i in range(future_days):
        future_date =  proc_date+datetime.timedelta(days=i)
        future_date_str = future_date.strftime("%Y-%m-%d")

        if not future_date_str in data_check_dic.keys():
            continue
        future_data_folder_path = "Tmp/"+future_date_str+"/"
        future_data_file = future_data_folder_path+"tweet_text"
        input_data_file_list.append(future_data_file)


    numtweets = 0
    numHashes = 10
    curtweets = 0

    print('Shingling tweets...')

    # The current shingle ID value to assign to the next new shingle we 
    # encounter. When a shingle gets added to the dictionary, we'll increment this
    # value.
    curShingleID = 0

    # Create a dictionary of the articles, mapping the article identifier (e.g., 
    # "t8470") to the list of shingle IDs that appear in the document.
    tweetAsShingleSets = {}
    tweetNames = []
    totalShingles = 0

    # if not os.path.exists(output_data_folder_path):
    #     os.makedirs(output_data_folder_path)

    # tweet_limit = 10000
    for input_data_file in input_data_file_list:
        with open(input_data_file, 'r', encoding='utf-8', errors='ignore') as file_in:
            for line in file_in:
                # tweet_limit -= 1
                # if tweet_limit < 0:
                #     break
                
                tweet_id = line.strip().split('\t')[0]
                try:
                    tweet_text =  line.strip().split('\t')[2]
                except:
                    continue
                tweet_token = tweet_text.strip().split()
                
                # Maintain a list of all tweet IDs.
                tweetNames.append(tweet_id)

                curtweets = curtweets + 1

                # 'shinglesInDoc' will hold all of the unique shingle IDs present in the 
                # current document. If a shingle ID occurs multiple times in the document,
                # it will only appear once in the set (this is a property of Python sets).
                shinglesInTweet = set()

                # For each word in the document...
                for index in range(0, len(tweet_token) - 2):
                    # Construct the shingle text by combining three words together.
                    shingle =tweet_token[index] + ' ' + tweet_token[index + 1] + ' ' + tweet_token[index + 2]
                    # shingle = tweet_text[index] + ' ' + tweet_text[index + 1] + ' ' + tweet_text[index + 2]
    

                    # Hash the shingle to a 32-bit integer.
                    crc = binascii.crc32(bytes(shingle, encoding="ascii")) & 0xffffffff

            
                    # Add the hash value to the list of shingles for the current document. 
                    # Note that set objects will only add the value to the set if the set 
                    # doesn't already contain it. 
                    shinglesInTweet.add(crc)

                # Store the completed list of shingles for this document in the dictionary.
                tweetAsShingleSets[tweet_id] = shinglesInTweet
    numtweets = curtweets


    # =============================================================================
    #                     Define Triangle Matrices
    # =============================================================================

    # Define virtual Triangle matrices to hold the similarity values. For storing
    # similarities between pairs, we only need roughly half the elements of a full
    # matrix. Using a triangle matrix requires less than half the memory of a full
    # matrix, and can protect the programmer from inadvertently accessing one of
    # the empty/invalid cells of a full matrix.

    # Calculate the number of elements needed in our triangle matrix
    numElems = int(numtweets * (numtweets - 1) / 2)

    # Initialize two empty lists to store the similarity values. 
    # 'JSim' will be for the actual Jaccard Similarity values. 
    # 'estJSim' will be for the estimated Jaccard Similarities found by comparing
    # the MinHash signatures.
    JSim = [0 for x in range(numElems)]
    estJSim = [0 for x in range(numElems)]

    def getTriangleIndex(i, j):
    # If i == j that's an error.
        if i == j:
            sys.stderr.write("Can't access triangle matrix with i == j")
            sys.exit(1)
        # If j < i just swap the values.
        if j < i:
            i, j = j, i
            # temp = i
            # i = j
            # j = temp

        # Calculate the index within the triangular array.
        # This fancy indexing scheme is taken from pg. 211 of:
        # http://infolab.stanford.edu/~ullman/mmds/ch6.pdf
        # But I adapted it for a 0-based index.
        # Note: The division by two should not truncate, it
        #       needs to be a float. 
        k = int(i * (numtweets - (i + 1) / 2.0) + j - i) - 1

        return k

    # =============================================================================
    #                 Generate MinHash Signatures
    # =============================================================================
    print ('Generating random hash functions...')

    # Record the maximum shingle ID that we assigned.
    maxShingleID = 2**32-1

    # We need the next largest prime number above 'maxShingleID'.
    # I looked this value up here: 
    # http://compoasso.free.fr/primelistweb/page/prime/liste_online_en.php
    nextPrime = 4294967311


    # Our random hash function will take the form of:
    #   h(x) = (a*x + b) % c
    # Where 'x' is the input value, 'a' and 'b' are random coefficients, and 'c' is
    # a prime number just greater than maxShingleID.

    # Generate a list of 'k' random coefficients for the random hash functions,
    # while ensuring that the same value does not appear multiple times in the 
    # list.
    def pickRandomCoeffs(k):
        # Create a list of 'k' random values.
        randList = []
    
        while k > 0:
            # Get a random shingle ID.
            randIndex = random.randint(0, maxShingleID) 
    
            # Ensure that each random number is unique.
            while randIndex in randList:
                randIndex = random.randint(0, maxShingleID) 
        
            # Add the random number to the list.
            randList.append(randIndex)
            k = k - 1
        
        return randList

    # For each of the 'numHashes' hash functions, generate a different coefficient 'a' and 'b'.   
    coeffA = pickRandomCoeffs(numHashes)
    coeffB = pickRandomCoeffs(numHashes)


    print ('Generating MinHash signatures for all documents...')

    # List of documents represented as signature vectors
    signatures = []

    # Rather than generating a random permutation of all possible shingles, 
    # we'll just hash the IDs of the shingles that are *actually in the document*,
    # then take the lowest resulting hash code value. This corresponds to the index 
    # of the first shingle that you would have encountered in the random order.

    # For each tweet...
    for tweetID in tweetNames:
    
        # Get the shingle set for this document.
        shingleIDSet = tweetAsShingleSets[tweetID]
    
        # The resulting minhash signature for this document. 
        signature = []
    
        # For each of the random hash functions...
        for i in range(0, numHashes):
        
            # For each of the shingles actually in the document, calculate its hash code
            # using hash function 'i'. 
        
            # Track the lowest hash ID seen. Initialize 'minHashCode' to be greater than
            # the maximum possible value output by the hash.
            minHashCode = nextPrime + 1
        
            # For each shingle in the document...
            for shingleID in shingleIDSet:
                # Evaluate the hash function.
                hashCode = (coeffA[i] * shingleID + coeffB[i]) % nextPrime 
        
                # Track the lowest hash code seen.
                if hashCode < minHashCode:
                    minHashCode = hashCode

            # Add the smallest hash code value as component number 'i' of the signature.
            signature.append(minHashCode)
        # Store the MinHash signature for this document.
        signatures.append(signature)


    # =============================================================================
    #                     Compare All Signatures
    # =============================================================================  
    print ('Comparing all signatures...')

    # Creates a N x N matrix initialized to 0.
    # For each of the test documents...
    for i in range(0, numtweets):
   
        # Get the MinHash signature for document i.
        signature1 = signatures[i]
        # if i%100==0:
        #     print(i)
        
        # For each of the other test documents...
        for j in range(i + 1, numtweets):
        
            # Get the MinHash signature for document j.
            try:
                signature2 = signatures[j]
            except:
                print("warning on comparing signatures")
                continue
        
            count = 0
            # Count the number of positions in the minhash signature which are equal.
            for k in range(0, numHashes):
                count = count + (signature1[k] == signature2[k])
        
            # Record the percentage of positions which matched.    
            estJSim[getTriangleIndex(i, j)] = (count / numHashes)


    # =============================================================================
    #                   Display Similar Document Pairs
    # =============================================================================  




    threshold = 0.5
    # For each of the document pairs...
    with open(output_data_folder_path+'cluster','w') as same_file, open(output_data_folder_path+'non_cluster','w') as unique_file:
        same_dict = {}
        same_list = []
        for i in range(0, numtweets):
            if not tweetNames[i] in same_list:
                value = ''
                for j in range(i + 1, numtweets):
                    if not tweetNames[j] in same_list:
                        # Retrieve the estimated similarity value for this pair.
                        estJ = estJSim[getTriangleIndex(i, j)]
                
                        # If the similarity is above the threshold...
                        if estJ > threshold:

                            if tweetNames[i] in same_dict.keys():
                                value = same_dict[tweetNames[i]]+':'+str(tweetNames[j])
                                same_dict[tweetNames[i]] = value
                                same_list.append(tweetNames[j])
                            else:
                                value = str(tweetNames[j])
                                same_dict[tweetNames[i]] = value
                                same_list.append(tweetNames[i])
                                same_list.append(tweetNames[j])
                # unique
                if not value:
                    unique_file.write(str(tweetNames[i])+':')             
                            

        
        for k, v in same_dict.items():
            same_file.write(k+':'+v+'\n')


    proc_date = proc_date+datetime.timedelta(days=1)
    if proc_date == end_date_datetime:
        break

2022-01-01
Shingling tweets...
Generating random hash functions...
Generating MinHash signatures for all documents...
Comparing all signatures...


In [22]:
import os
import json
import datetime




start_date = "20220101"
end_date = "20220102"
start_date_datetime = datetime.datetime.strptime(start_date, '%Y%m%d')
end_date_datetime = datetime.datetime.strptime(end_date, '%Y%m%d')
proc_date = start_date_datetime
duration = 300   #

data_check_list = os.listdir("Data/")
data_check_dic = {i:1 for i in data_check_list}

for _ in range(duration):
    # process the data in this date
    proc_date_str = proc_date.strftime("%Y-%m-%d")

    input_data_folder_path = "Tmp/"+proc_date_str+"/"
    output_data_folder_path =  "Cluster/"+proc_date_str+"/"

    if not proc_date_str in data_check_dic.keys():
        proc_date = proc_date+datetime.timedelta(days=1)
        if proc_date == end_date_datetime:
            break
        continue

    if not os.path.exists(output_data_folder_path):
        os.makedirs(output_data_folder_path)



    input_data_cluster = input_data_folder_path+'cluster'
    input_data_text = input_data_folder_path+'tweet_text'
    output_data_file = output_data_folder_path+"tweets.txt"

    cluster_dic = {}
    with open (input_data_cluster, 'r') as file_in:
        for line in file_in:
            line_split = line.strip().split(":")

            cluster_dic[line_split[0]] = len(line_split)

    with open(input_data_text, 'r') as file_in, \
        open(output_data_file, 'w') as file_out:
        for line in file_in:
            line_split = line.strip().split("\t")
            tweet_id = line_split[0]
            user_id = line_split[1]
            tweet_text = line_split[2]

            if tweet_id in cluster_dic.keys():
                file_out.write('N'+'\t')
                file_out.write(str(cluster_dic[tweet_id])+'\t')
                file_out.write(tweet_id+'\t')
                file_out.write(user_id+'\t')
                file_out.write(tweet_text+'\n')
                file_out.flush()

    proc_date = proc_date+datetime.timedelta(days=1)
    if proc_date == end_date_datetime:
        break




In [23]:
import os
import json
import datetime




start_date = "20220101"
end_date = "20220102"
start_date_datetime = datetime.datetime.strptime(start_date, '%Y%m%d')
end_date_datetime = datetime.datetime.strptime(end_date, '%Y%m%d')
proc_date = start_date_datetime
duration = 300   #

data_check_list = os.listdir("Data/")
data_check_dic = {i:1 for i in data_check_list}


for _ in range(duration):
    # process the data in this date
    proc_date_str = proc_date.strftime("%Y-%m-%d")

    input_data_tmp_path = "Tmp/"+proc_date_str+"/"
    input_data_folder_path = "Cluster/"+proc_date_str+"/"
    output_data_folder_path =  "Label/"+proc_date_str+"/"

    if not proc_date_str in data_check_dic.keys():
        proc_date = proc_date+datetime.timedelta(days=1)
        if proc_date == end_date_datetime:
            break
        continue

    if not os.path.exists(output_data_folder_path):
        os.makedirs(output_data_folder_path)

    input_data_text = input_data_tmp_path+"tweet_text"
    input_data_cluster = input_data_tmp_path+'cluster'
    input_data_label = input_data_folder_path+'tweets.txt'
    output_data_file = output_data_folder_path+"labeled_tweets.txt"

    cluster_reverse = {}
    with open(input_data_cluster, 'r') as file_in:
        for line in file_in:
            line_split = line.strip().split(":")
            for lc in line_split:
                cluster_reverse[lc] = line_split[0]

    label_dic = {}
    with open(input_data_label, 'r') as file_in:
        for line in file_in:
            line_split = line.strip().split("\t")
            if not line_split[0] == 'N':
                label_dic[line_split[2]] = line_split[0]

    with open(input_data_text, 'r') as file_in, \
        open(output_data_file, 'w') as file_out:
        for line in file_in:
            line_split = line.strip().split("\t")
            tweet_id = line_split[0]
            user_id = line_split[1]
            tweet_text = line_split[2]

            if not tweet_id in cluster_reverse.keys():
                continue
            cluster_id = cluster_reverse[tweet_id]

            if not cluster_id in label_dic.keys():
                continue

            label = label_dic[cluster_id]
      

            file_out.write(tweet_id+'\t')
            file_out.write(user_id+'\t')
            file_out.write(tweet_text+'\t')
            file_out.write(label+'\n')
            file_out.flush()

    proc_date = proc_date+datetime.timedelta(days=1)
    if proc_date == end_date_datetime:
        break



In [24]:
import os
import json
import datetime
from tool.Twitter import Tweet

start_date = "20220101"
end_date = "20220102"
start_date_datetime = datetime.datetime.strptime(start_date, '%Y%m%d')
end_date_datetime = datetime.datetime.strptime(end_date, '%Y%m%d')
proc_date = start_date_datetime
duration = 300   # t

data_check_list = os.listdir("Data/")
data_check_dic = {i:1 for i in data_check_list}

for _ in range(duration):
    # process the data in this date
    proc_date_str = proc_date.strftime("%Y-%m-%d")

    input_data_folder_path = "Data/"+proc_date_str+"/"
    output_data_folder_path =   "Tmp/"+proc_date_str+"/"


    if not proc_date_str in data_check_dic.keys():
        proc_date = proc_date+datetime.timedelta(days=1)
        if proc_date == end_date_datetime:
            break
        continue
    
    if not os.path.exists(output_data_folder_path):
        os.makedirs(output_data_folder_path)

    output_data_file = output_data_folder_path+"tweet_feature"
    with open(output_data_file, 'w', encoding='utf-8') as file_out:


        for filename in os.listdir(input_data_folder_path):
            input_data_path = input_data_folder_path+filename

            with open(input_data_path, 'r', encoding='utf-8', errors='ignore') as file_in:
                
                for line in file_in:
                    try:
                        tweet = json.loads(line)
                        tweet_obj = Tweet(tweet)
                    except:
                        print(tweet)
                        continue
                    
                    if not tweet_obj.is_en():
                        continue

                    tweet_id = tweet_obj.get_id()
                    user_id = tweet_obj.user.id_str
                    feature_list = tweet_obj.get_tweet_features()
                    file_out.write(tweet_id+'\t'+user_id+'\t')

                    for f in feature_list:
                        file_out.write(str(f))
                        file_out.write('\t')
                    file_out.write('\n')
                    file_out.flush()
                    
    proc_date = proc_date+datetime.timedelta(days=1)
    if proc_date == end_date_datetime:
        break

IsADirectoryError: [Errno 21] Is a directory: 'Data/2022-01-01/.ipynb_checkpoints'